In [ ]:
import requests
from tqdm import tqdm

In [ ]:
def download_file(url, destination):
    response = requests.get(url, stream=True)
    total_size_in_bytes = int(response.headers.get('content-length', 0))
    block_size = 1024 # 1 Kibibyte
    progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)
    
    with open(destination, 'wb') as file:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            file.write(data)
    progress_bar.close()
    
    if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
        print("ERROR, something went wrong")
    else:
        print(f"Download completed. File saved as {destination}")

# # VOC 2007 训练/验证数据
# download_file(
#     "http://pjreddie.com/media/files/VOCtest_06-Nov-2007.tar",
#     "VOCtrainval_06-Nov-2007.tar"
# )

# # VOC 2007 测试数据
# download_file(
#     "http://pjreddie.com/media/files/VOCtest_06-Nov-2007.tar",
#     "VOCtest_06-Nov-2007.tar"
# )

#VOC 2012 训练数据
download_file(
    "http://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar",
    "VOCtrainval_11-May-2012.tar"
)
download_file(
    "http://pjreddie.com/media/files/VOC2012test.tar",
    "VOC2012test.tar"
)

In [ ]:
import os
import shutil
import numpy as np

# 配置路径
path_2007 = r'D:\复旦\研究生\研一下\神经网络与深度学习\Homework2\VOCtrainval_06-Nov-2007\VOCdevkit\VOC2007'
path = r'D:\复旦\研究生\研一下\神经网络与深度学习\Homework2\VOCtrainval_11-May-2012\VOCdevkit\VOC2012'
source_image_dir = path + r'\JPEGImages'  # 原始图片存储路径
source_xml_dir = path + r'\Annotations'  # 原始XML存储路径
train_image_dir = path + r'\train\images'  # 训练集图片存储路径
train_xml_dir = path + r'\train\gt'  # 训练集XML存储路径
val_image_dir = path + r'\val\images' # 验证集图片存储路径
val_xml_dir = path + r'\val\gt'  # 验证集XML存储路径
val_ratio = 0.2  # 验证集比例

# 获取图片和XML文件
image_files = {f.split('.')[0] for f in os.listdir(source_image_dir) if f.endswith('.jpg')}
xml_files = {f.split('.')[0] for f in os.listdir(source_xml_dir) if f.endswith('.xml')}

# 确保图片和XML文件名匹配
matched_files = image_files.intersection(xml_files)
matched_files = list(matched_files)  # 转换为列表以进行索引操作

# 随机划分训练集和验证集
np.random.seed(42)
np.random.shuffle(matched_files)
split_point = int(len(matched_files) * (1 - val_ratio))
train_files = matched_files[:split_point]
val_files = matched_files[split_point:]

# 创建目录
os.makedirs(train_image_dir, exist_ok=True)
os.makedirs(train_xml_dir, exist_ok=True)
os.makedirs(val_image_dir, exist_ok=True)
os.makedirs(val_xml_dir, exist_ok=True)

# 复制文件到新目录
for filename in train_files:
    shutil.copy(os.path.join(source_image_dir, filename + '.jpg'), os.path.join(train_image_dir, filename + '.jpg'))
    shutil.copy(os.path.join(source_xml_dir, filename + '.xml'), os.path.join(train_xml_dir, filename + '.xml'))

for filename in val_files:
    shutil.copy(os.path.join(source_image_dir, filename + '.jpg'), os.path.join(val_image_dir, filename + '.jpg'))
    shutil.copy(os.path.join(source_xml_dir, filename + '.xml'), os.path.join(val_xml_dir, filename + '.xml'))

print("Files have been successfully copied to the training and validation directories.")

In [9]:
import os
import xml.etree.ElementTree as ET
import json
from collections import defaultdict

class VocToCoco:

    def __init__(self, voc_gt_dir: str, output_coco_path: str) -> None:
        self.voc_gt_dir = voc_gt_dir
        self.output_coco_path = output_coco_path
        self.categories_count = 1
        self.images = []
        self.categories = {}
        self.annotations = []
        self.data = defaultdict(list)

    # 图片处理
    def images_handle(self, root: ET.Element, img_id: int) -> None:
        filename = root.find('filename').text.strip()
        width = int(root.find('size').find('width').text)
        height = int(root.find('size').find('height').text)

        self.images.append({
            'id': int(img_id),
            'file_name': filename,
            'height': height,
            'width': width,
        })

    # 标签转换
    def categories_handle(self, category: str) -> None:
        if category not in self.categories:
            self.categories[category] = {'id': len(self.categories) + 1, 'name': category}

    # 标注转换
    def annotations_handle(self, bbox: ET.Element, img_id: int, category: str) -> None:
        x1 = int(float(bbox.find('xmin').text))
        y1 = int(float(bbox.find('ymin').text))
        x2 = int(float(bbox.find('xmax').text))
        y2 = int(float(bbox.find('ymax').text))

        width = x2 - x1
        height = y2 - y1
        area = width * height

        self.annotations.append({
            'id': self.categories_count,
            'image_id': int(img_id),
            'category_id': self.categories[category].get('id'),
            'bbox': [x1, y1, width, height],
            'area': area,
            'iscrowd': 0
        })
        self.categories_count += 1

    def parse_voc_annotation(self) -> None:
        for img_id, filename in enumerate(os.listdir(self.voc_gt_dir), 1):
            xml_file = os.path.join(self.voc_gt_dir, filename)
            tree = ET.parse(xml_file)
            root = tree.getroot()

            self.images_handle(root, img_id)

            for obj in root.iter('object'):
                category = obj.find('name').text
                self.categories_handle(category)

                bbox = obj.find('bndbox')
                self.annotations_handle(bbox, img_id, category)

        self.data['images'] = self.images
        self.data['categories'] = list(self.categories.values())
        self.data['annotations'] = self.annotations

        with open(self.output_coco_path, 'w') as f:
            json.dump(self.data, f, ensure_ascii=False, indent=2)

In [ ]:
# 将训练集转化为coco格式
voc_gt_dir = path + r'\train\gt'
img_dir = path + r'\train\images'
output_coco_path = path + r'\train\train.json'

voc2coco = VocToCoco(voc_gt_dir, output_coco_path)
voc2coco.parse_voc_annotation()


# 将验证集转化为coco格式
voc_gt_dir = path + r'\val\gt'
img_dir = path + r'\val\images'
output_coco_path = path + r'\val\val.json'

voc2coco = VocToCoco(voc_gt_dir, output_coco_path)
voc2coco.parse_voc_annotation()


# 将测试集转化为coco格式
voc_gt_dir = r'D:\复旦\研究生\研一下\神经网络与深度学习\Homework2\VOC2012test\VOCdevkit\VOC2012\Annotations'
img_dir = r'D:\复旦\研究生\研一下\神经网络与深度学习\Homework2\VOC2012test\VOCdevkit\VOC2012\JPEGImages'
output_coco_path = r'D:\复旦\研究生\研一下\神经网络与深度学习\Homework2\VOC2012test\VOCdevkit\VOC2012\test.json'

voc2coco = VocToCoco(voc_gt_dir, output_coco_path)
voc2coco.parse_voc_annotation()

In [1]:
path = r'D:\复旦\研究生\研一下\神经网络与深度学习\Homework2\VOCtrainval_11-May-2012\VOCdevkit\VOC2012'
# 将验证集转化为coco格式
voc_gt_dir = path + r'\val\gt'
img_dir = path + r'\val\images'
output_coco_path = path + r'\val\val.json'

voc2coco = VocToCoco(voc_gt_dir, output_coco_path)
voc2coco.parse_voc_annotation()

NameError: name 'path' is not defined

In [8]:
int(45.70000076293945)

45